In [1]:
import os
import pathlib
from urllib.request import urlretrieve
from time import sleep
import math as math
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as ticker

import sys
sys.path.append('/code')
#from fdmonitor.cont_futures import get_roll_date, create_continuous_future

%matplotlib inline

In [2]:
quandl.ApiConfig.api_key = os.environ['QUANDL_API_KEY']

In [2]:
cme_metadata = pd.read_csv('CME_metadata.csv')
cme_metadata.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=https...",2018-11-14 01:09:43,2015-02-12,2018-11-13


In [17]:
# Generate list of contract codes for ES
es_contract_codes = cme_metadata[cme_metadata['code'].str.startswith('ES')]['code'].tolist()

In [3]:
DATA_DIRNAME = pathlib.Path('/code').resolve() / 'data'

In [34]:
API_URL = f'http://www.quandl.com/api/v3/datasets/CME/ESZ2018.csv?auth_token={quandl.ApiConfig.api_key}'
API_URL

'http://www.quandl.com/api/v3/datasets/CME/ESZ2018.csv?auth_token=TFddxxSjUbLtV482Dq7Q'

In [55]:
def download_contract_from_quandl(exchange_code, contract_code, data_directory):
    API_URL = f'http://www.quandl.com/api/v3/datasets/{exchange_code}/{contract_code}.csv' \
            f'?auth_token={quandl.ApiConfig.api_key}&order=asc'
    CONTRACT_DATA_DIRNAME = data_directory / 'raw_csv' / 'futures' / f'{contract_code[0:2]}' / 'daily'
    
    CONTRACT_DATA_DIRNAME.mkdir(parents=True, exist_ok=True)
    os.chdir(CONTRACT_DATA_DIRNAME)
    urlretrieve(API_URL, f'{contract_code}.csv')
    sleep(0.03) # API Rate Limit is 300 calls per 10 seconds, 2000 calls per 10 minutes
    
    return API_URL

for contract_code in es_contract_codes:
    print (f'Downloading Contract: {contract_code}')
    download_contract_from_quandl('CME', contract_code, DATA_DIRNAME)

In [4]:
cme_futures = pd.read_csv('CME_futures.csv')
cme_futures.set_index('Futures Symbol', inplace=True)
cme_futures.head()

,Exchange,Name,Hold Roll Cycle,Priced Roll Cycle,CME Months,Start Year
Futures Symbol,,,,,,
AD,CME,Australian Dollar Futures,HMUZ,HMUZ,HMUZ,1987
BO,CBT,Soybean Oil Futures,Z,FHKNQUVZ,FHKNQUVZ,1962
BP,CME,British Pound Futures,HMUZ,HMUZ,HMUZ,1976
C,CBT,Corn Futures,Z,HKNUZ,HKNUZ,1960
CD,CME,Canadian Dollar Futures,HMUZ,HMUZ,HMUZ,1978


In [5]:
futures_symbol = 'FC'
futures_name = cme_futures.loc[futures_symbol, 'Name']
month_codes = cme_futures.loc[futures_symbol, 'Priced Roll Cycle']
first_month_code = month_codes[0]
last_month_code = month_codes[-1]

length = (cme_metadata['code'].str.len() == len(futures_symbol) + 5)
#starts_with = cme_metadata['code'].str.startswith(futures_symbol+first_month_code)
#results = cme_metadata[length & starts_with]
#first_year = int(results.iloc[0,0][-4:])
first_year = int(cme_futures.loc[futures_symbol, 'Start Year'])
starts_with = cme_metadata['code'].str.startswith(futures_symbol+last_month_code)
results = cme_metadata[length & starts_with]
last_year = int(results.iloc[-1,0][-4:])

#first_year = 1962
#ticker = futures_symbol
CME_months = month_codes
CME_years = [i for i in range(first_year, last_year + 1)]
first_year

1982

In [9]:
main = create_continuous_future(futures_symbol, DATA_DIRNAME, CME_months, CME_years,
                               'open_interest', False)
main.tail()

,Open,High,Low,Close,Volume,Open Interest
index,,,,,,
2018-11-23,2650.00,2656.25,2626.00,2628.5,827480.0,2793684.0
2018-11-26,2633.50,2676.25,2631.50,2661.0,1337376.0,2792725.0
2018-11-27,2661.00,2686.75,2655.25,2686.0,1466895.0,2783014.0
2018-11-28,2686.25,2745.00,2681.00,2741.0,1805261.0,2763027.0
2018-11-29,2738.00,2754.50,2723.25,2742.0,1468632.0,2773110.0


In [17]:
pd.options.display.max_rows = 999
#CME_months = ['H','M','U','Z']
#CME_years = [i for i in range(1999,2019)]

#ticker = 'ES'
roll_rule = 'open_interes'
is_backadjusted = False
df_dict = {}
contracts = []
roll_dates = []
start = np.datetime64('1900-01-01')

CONTRACT_DATA_DIRNAME = DATA_DIRNAME / 'raw_csv' / 'futures' / f'{futures_symbol}' / 'daily'

for i in range(0,len(CME_months)*len(CME_years)):
    month = CME_months[i % len(CME_months)]
    year = CME_years[math.floor(i/len(CME_months))]
    contract = futures_symbol + month + str(year)
    contracts.append(contract)
    current_df = pd.read_csv(CONTRACT_DATA_DIRNAME / f'{futures_symbol}{month}{year}.csv')
    current_df.set_index('Date', inplace=True)
    current_df.index = pd.to_datetime(current_df.index)
    current_df['Close'] = np.where(current_df['Last'].isnull(), current_df['Settle'], current_df['Last'] )
    current_df['Open Interest'] = current_df[current_df.columns[7]]
    current_df = current_df[['Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]
    df_dict[contract]= current_df
    
    if i > 0:
        roll_date = get_roll_date(i-1, df_dict, contracts, roll_rule, start)
        #print(roll_date)
        if start != roll_date:
            roll_dates.append(roll_date)
            start = roll_date
        else:
            del roll_dates[-1]
            break

cont = df_dict[contracts[0]]
start = roll_dates[0]
switch_post = pd.DataFrame(columns = ('Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest'))
switch_pre = pd.DataFrame(columns = ('Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest'))
switch_pre = switch_pre.append(cont.iloc[cont.index.get_loc(start,method='bfill')])

for i in range(1,len(roll_dates)+1):
    cur = df_dict[contracts[i]]
    print(contracts[i], start)
    cur = cur[cur.index.values >= start]
    #roll_date = switch_pre.iloc[i-1].name
    #cur = cur[cur.index.values > roll_date]
    #switch_post = switch_post.append(cur.loc[roll_date])
    if len(cur.index) != 0:
        switch_post = switch_post.append(cur.iloc[0])
        cont = cont.append(cur)
        if i != len(roll_dates):
            start = roll_dates[i]
            switch_pre = switch_pre.append(cur.iloc[cur.index.get_loc(start,method='bfill')])

#switch_pre = switch_pre.drop(switch_pre.index[len(switch_pre)-1])
switch_post['Change'] = switch_post['Close'].values - switch_pre['Close'].values
switch_post = pd.DataFrame(switch_post['Change'])

main = cont.merge(switch_post,how='left',left_index=True,right_index=True)
main['Change'].fillna(0, inplace=True)
main = main.reset_index().drop_duplicates(subset='index',keep='last').set_index('index')
main['BAdj'] = main.loc[::-1, 'Change'].cumsum()[::-1]
main['BAdj'] = main['BAdj'].shift(-1)
main['BAdj'].fillna(0, inplace=True)


if is_backadjusted:
    main['Open'] = main['Open'] + main['BAdj']
    main['High'] = main['High'] + main['BAdj']
    main['Low'] = main['Low'] + main['BAdj']
    main['Close'] = main['Close'] + main['BAdj']

#main.drop(columns=['Change','BAdj'], inplace=True)
main
#main = main.fillna(0)
#main
#switch_pre, switch_post
#~current_df['Change'].isnull()
#df_dict['ESH2018']


FCH1982 1982-01-20T00:00:00.000000000
FCJ1982 1982-03-19T00:00:00.000000000
FCK1982 1982-04-20T00:00:00.000000000
FCQ1982 1982-05-20T00:00:00.000000000
FCU1982 1982-08-20T00:00:00.000000000
FCV1982 1982-09-20T00:00:00.000000000
FCX1982 1982-10-20T00:00:00.000000000
FCF1983 1982-11-19T00:00:00.000000000
FCH1983 1983-01-20T00:00:00.000000000
FCJ1983 1983-03-18T00:00:00.000000000
FCK1983 1983-04-20T00:00:00.000000000
FCQ1983 1983-05-20T00:00:00.000000000
FCU1983 1983-08-19T00:00:00.000000000
FCV1983 1983-09-20T00:00:00.000000000
FCX1983 1983-10-20T00:00:00.000000000
FCF1984 1983-11-18T00:00:00.000000000
FCH1984 1984-01-20T00:00:00.000000000
FCJ1984 1984-03-20T00:00:00.000000000
FCK1984 1984-04-19T00:00:00.000000000
FCQ1984 1984-05-18T00:00:00.000000000
FCU1984 1984-08-20T00:00:00.000000000
FCV1984 1984-09-20T00:00:00.000000000
FCX1984 1984-10-19T00:00:00.000000000
FCF1985 1984-11-20T00:00:00.000000000
FCH1985 1985-01-18T00:00:00.000000000
FCJ1985 1985-03-20T00:00:00.000000000
FCK1985 1985

FCF2014 2013-11-22T00:00:00.000000000
FCH2014 2014-01-31T00:00:00.000000000
FCJ2014 2014-03-28T00:00:00.000000000
FCK2014 2014-04-21T00:00:00.000000000
FCQ2014 2014-05-23T00:00:00.000000000
FCU2014 2014-08-29T00:00:00.000000000
FCV2014 2014-09-26T00:00:00.000000000
FCX2014 2014-10-31T00:00:00.000000000
FCF2015 2014-11-21T00:00:00.000000000
FCH2015 2015-01-30T00:00:00.000000000
FCJ2015 2015-03-27T00:00:00.000000000
FCK2015 2015-05-01T00:00:00.000000000
FCQ2015 2015-05-22T00:00:00.000000000
FCU2015 2015-08-28T00:00:00.000000000
FCV2015 2015-09-25T00:00:00.000000000
FCX2015 2015-10-30T00:00:00.000000000
FCF2016 2015-11-20T00:00:00.000000000
FCH2016 2016-01-29T00:00:00.000000000
FCJ2016 2016-03-28T00:00:00.000000000
FCK2016 2016-04-29T00:00:00.000000000
FCQ2016 2016-05-27T00:00:00.000000000
FCU2016 2016-08-26T00:00:00.000000000
FCV2016 2016-09-30T00:00:00.000000000
FCX2016 2016-10-28T00:00:00.000000000
FCF2017 2016-11-18T00:00:00.000000000
FCH2017 2017-01-27T00:00:00.000000000
FCJ2017 2017

,Open,High,Low,Close,Volume,Open Interest,Change,BAdj
index,,,,,,,,
1981-02-03,73.000,73.000,72.900,72.900,2.0,2.0,0.000,-110.670
1981-02-04,73.250,73.250,72.900,72.900,0.0,2.0,0.000,-110.670
1981-02-05,73.500,73.500,72.900,72.900,0.0,2.0,0.000,-110.670
1981-02-06,74.400,74.400,74.200,74.200,0.0,2.0,0.000,-110.670
1981-02-09,75.300,75.300,75.300,75.300,0.0,2.0,0.000,-110.670
1981-02-10,75.300,75.300,75.300,75.300,0.0,2.0,0.000,-110.670
1981-02-11,74.600,74.600,74.600,74.600,0.0,2.0,0.000,-110.670
1981-02-12,74.120,74.120,74.120,74.120,0.0,2.0,0.000,-110.670
1981-02-13,74.500,74.500,74.500,74.500,0.0,2.0,0.000,-110.670


In [14]:
df_dict['FCJ2003']

,Open,High,Low,Close,Volume,Open Interest
Date,,,,,,
2002-04-26,77.00,77.00,77.00,77.00,0.0,0.0
2002-04-29,76.00,76.00,76.00,76.00,2.0,2.0
2002-04-30,76.30,76.30,76.30,76.30,0.0,2.0
2002-05-01,76.40,76.40,76.40,76.40,0.0,2.0
2002-05-02,76.10,76.10,76.10,76.10,0.0,2.0
2002-05-03,76.30,76.30,76.30,76.30,0.0,2.0
2002-05-06,77.80,77.80,77.80,77.80,0.0,2.0
2002-05-07,78.00,78.00,78.00,78.00,0.0,2.0
2002-05-08,78.25,78.25,77.60,77.60,2.0,4.0


In [9]:
def get_roll_date(i, df_dict, contracts, roll_rule, start=np.datetime64('1900-01-01')):
    cur_con = df_dict[contracts[i]]

    if roll_rule == 'open_interest':
        try:
            next_con = df_dict[contracts[i+1]]
            sub = next_con.merge(cur_con,how='inner',left_index=True,right_index=True)
            sub = sub[sub.index.values > start]
            sub = sub[sub['Open Interest_x'] > sub['Open Interest_y']]
            if sub.empty:
                start = cur_con.index.values[len(cur_con)-1]
            else:
                start = sub.index.values[0]
        except:
            return np.datetime64('1900-01-01')
    elif roll_rule == 'exp_month':
        #print(contracts[i])
        start = cur_con.index.values[len(cur_con)-1]
        starts = cur_con.index - pd.offsets.MonthBegin(1)
        start = starts.values[len(cur_con)-1]
    else:
        start = cur_con.index.values[len(cur_con)-1]
    
    return start

get_roll_date(0, df_dict, contracts, 'open_interest')
    

numpy.datetime64('1981-07-21T00:00:00.000000000')

In [7]:
def plot_data(df, ax, fmt):
    ax.patch.set(facecolor='w', edgecolor='k', linewidth=1.0)
    df['timestamp'] = df.index.to_pydatetime()
    df['ema'] = df['Close'].ewm(span=20, min_periods=20).mean()
    df = df[-80:]
    df = df.reset_index(drop=True)
    df['x'] = df.index
    
    subset = df[['x','Open','High','Low','Close']]
    tuples = [tuple(x) for x in subset.values]
    _ = candlestick_ohlc(ax, tuples, width=.5, colorup='g', colordown='r', alpha=1)
    _ = ax.plot(df['x'].tolist(), df['ema'].tolist(), color='blue', lw=0.5)
    
    def formatter(x, pos=None):
        thisind = np.clip(int(x + 0.5), 0, len(df.index) - 1)
        return df['timestamp'][thisind].strftime(fmt)

    ax.xaxis.set_major_formatter(ticker.FuncFormatter(formatter))
    xticklabels = df['x'].iloc[::11].tolist()
    ax.xaxis.set_major_locator(ticker.FixedLocator(xticklabels))
    
    for line in ax.xaxis.get_ticklines():
        line.set_markersize(5)
    for line in ax.yaxis.get_ticklines():
        line.set_markersize(5)
    for line in ax.yaxis.get_gridlines():
        line.set(color='gray', ls='dotted', alpha=0.5)

#plot_data(main, ax_daily)

In [8]:
def draw_charts(data, plot_title, pdf_pages=None):
    fig = plt.figure(figsize=(16,12))
    ax_daily = plt.subplot2grid((12,16), (0,0), colspan=16, rowspan=7)
    ax_weekly = plt.subplot2grid((12,16), (7,0), colspan=8, rowspan=5)
    ax_monthly = plt.subplot2grid((12,16), (7,8), colspan=8, rowspan=5)

    ax_daily.set_title(plot_title, fontsize=20)

    plot_data(data, ax_daily, '%b %-d')
    df_weekly = data.resample('W').agg({
        'Open' : 'first',
        'High' : max,
        'Low' : min,
        'Close' : 'last',
        'Volume' : sum,
        'Open Interest' : 'last'
    })
    plot_data(df_weekly, ax_weekly, "%b %y'")
    df_monthly = data.resample('M').agg({
        'Open' : 'first',
        'High' : max,
        'Low' : min,
        'Close' : 'last',
        'Volume' : sum,
        'Open Interest' : 'last'
    })
    plot_data(df_monthly, ax_monthly, '%b %Y')

    plt.tight_layout(pad=2.5)
    
    if pdf_pages:
        pdf_pages.savefig(fig)
    else:
        plt.show()

    plt.close(fig)

In [9]:
roll_rules = ['expiration', 'open_interest', 'exp_month']
true_false = [False, True]

pdf_pages = PdfPages('ES.pdf')

for roll_rule in roll_rules:
    for tf in true_false:
        data = create_continuous_future(futures_symbol, DATA_DIRNAME, CME_months, CME_years,
                                        roll_rule, tf)
        plot_title = f'{futures_name} ({futures_symbol}) \n Roll: {roll_rule}'
        if tf:
            plot_title = plot_title + ', backadjusted'
        draw_charts(data, plot_title, pdf_pages)
pdf_pages.close()